In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

DIMENSION = (256, 1600) # image dimension

# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util
import mask_conversion

---

### Preparations

In [ ]:
df = pd.read_csv('data/train.csv')

In [ ]:
util.isolate_single_defects(df)

In [ ]:
df.head(10)

In [ ]:
def count_defect_pixels(EncodedPixels):
    sum = 0
    for idx, pixel in enumerate(EncodedPixels.split()):
        if idx%2 != 0:
            sum += int(pixel)

    return sum

In [ ]:
df['DefectSizePixel'] = df.EncodedPixels.apply(lambda x: count_defect_pixels(x))

---

### Testing

In [ ]:
a = np.zeros((256,10))

encoded_pixels = '1 1000'
a
mask_conversion.decode_pixel(a.shape,1,encoded_pixels,image=False)

In [ ]:
mask_conversion.decode_pixel(a.shape,1,encoded_pixels,image=False)

In [ ]:
np.sum(mask_conversion.decode_pixel(DIMENSION, 1, encoded_pixels=df.EncodedPixels[2]))

---

### Convert Encoded Pixels to a Mask and Build CheckSum

In [ ]:
sums = []
for i in range(df.shape[0]):
    sum_ = int(np.sum(mask_conversion.decode_pixel(DIMENSION, 1, encoded_pixels=df.EncodedPixels.iloc[i])))
    sums.append(sum_)


In [ ]:
df['MaskSums'] = sums
df['CheckSum'] = df.DefectSizePixel - df.MaskSums
df.describe()

In [ ]:
classes = np.zeros(DIMENSION)
np.sum(mask_conversion.decode_pixel(classes, 1, encoded_pixels=df.EncodedPixels[2]))

---

### Encode Pixel

In [ ]:
a = np.identity(5)

encoded_pixels = '1 1000'
a

In [ ]:
mask_conversion.encode_pixel(a)

In [ ]:
df.EncodedPixels[0]

In [ ]:
mask1 = mask_conversion.decode_pixel(DIMENSION, class_id=1, 
                                      encoded_pixels=df.EncodedPixels[0], image=True)

In [ ]:
mask_conversion.encode_pixel(mask1, 1)